# Part 2: Graphical Interface and Zoning
This jupyter noteboo will focus on the following:
1. Using the `tkinter` library to create a graphical interface
2. Tracing a polygon to draw Voronoi cells
3. Calculate the Voronoi diagram to allow the construction of a netwokr without any interferances. Each Voronoi zone will be controlled by different frequencies to ensure the control and communication between the parcel distribution drones.

In [1]:
# importing
import tkinter as tk
from Trie import *
from math import sqrt

For this part, we will be using the trie graph to search our database. Look at the jupyter notebook `1. Indexing` for more details.

In [2]:
database = generate_database(100)
tree = database_to_trie(database)

The code below makes a window that allows us to search for all 'zones' that begin with a user entered text. On running the code, a window like the image below should open up.
![Search window](images/search_window.png)

In [3]:
def onButtonSearch():
    global results_listbox
    global textbox
    global tree
    results_listbox.delete(0, tk.END)
    print(tree)
    _, result = list(find_prefix(tree, textbox.get()))
    textbox.delete(0, tk.END)
    for e in result:
        results_listbox.insert(tk.END, e)

window = tk.Tk()

search_frame = tk.Frame(window, borderwidth=2)
search_frame.pack(side=tk.TOP)

textbox = tk.Entry(search_frame, width=32)
textbox.pack(side=tk.LEFT, padx=5, pady=5)

search_button = tk.Button(search_frame, text="Search", command=onButtonSearch, height="2")
search_button.pack(side=tk.RIGHT, padx=5, pady=5)

results_listbox = tk.Listbox(window)
results_listbox.pack(padx=5, pady=5)

window.mainloop()

Next, we make a canvas to draw the zones and point. The functions below will help in drawing points and polygons to split the 2D coordinate space.

In [4]:
marginx = 20
canvas_len = 600

def draw_cartesian(canvas):
    global marginx
    canvas.create_line(marginx, canvas_len, marginx + canvas_len, canvas_len, arrow="last")
    canvas.create_line(marginx, canvas_len, 20, 0, arrow="last")
    canvas.create_text(marginx, canvas_len+10, text="(0,0)")
    canvas.create_text(canvas_len+10, canvas_len+10, text="x")
    canvas.create_text(10, 5, text="y")

def canvas_point(P):
    '''Convert (x,y) coordinates to the canvas coordinates'''
    global marginx
    return P[0] + marginx, canvas_len - P[1]

def cross(u, v):
    w = [0, 0, 0]
    w[0] = u[1] * v[2] - u[2] * v[1]
    w[1] = u[2] * v[0] - u[0] * v[2]
    w[2] = u[0] * v[1] - u[1] * v[0]
    return w

def to_left(pxmin, pxmax, p):
    pxVec3D = [pxmax[0] - pxmin[0],pxmax[1] - pxmin[1],0]
    pVec3D = [p[0] - pxmax[0], p[1] - pxmax[1], 0]
    result = cross(pxVec3D, pVec3D)
    return (result[2] >= 0)

def get_x_min(poly):
    L = sorted(poly)
    return L[0]

def get_x_max(poly):
    L = sorted(poly)
    return L[-1]

def points_left(pxmin, pxmax, poly):
    L = []
    for p in poly:
        if to_left(pxmin, pxmax, p) and p != pxmax and p != pxmin:
            L.append(p)
    return L

def points_right(pxmin, pxmax, poly):
    L = []
    for p in poly:
        if to_left(pxmin, pxmax, p) == False and p != pxmax and p != pxmin:
            L.append(p)
    return L

def tri_polygone(poly):
    '''Returns the points of a polygon sorted
    in the correct order to draw a convex polygon'''
    if(len(poly) == 0):
        return []
    pxmin = get_x_min(poly)
    pxmax = get_x_max(poly)
    LG = sorted(points_left(pxmin, pxmax, poly), reverse=1)
    LD = sorted(points_right(pxmin, pxmax, poly))
    return [pxmin] + LD + [pxmax] + LG

def draw_polygon(canvas, points):
    '''Draws a polygon on the screen'''
    if(len(points) == 0):
        return
    temp = []
    for p in points:
        temp.append(canvas_point(p))
    poly = tri_polygone(temp)
    canvas.create_polygon(poly, fill="skyblue")
    for i in range(0, len(poly)):
        p1i = i % len(poly)
        p2i = (i + 1) % len(poly)
        canvas.create_line(poly[p1i], poly[p2i], fill="darkgreen")

In [5]:
window = tk.Tk()

canvas = tk.Canvas(window, width=620, height=620)
draw_cartesian(canvas)

p01 = (20,50)
p02 = (70,220)
p03 = (250,190)
p04 = (300,150)
p05 = (150,20)
poly0 = [p01, p02, p03, p04, p05]

p11 = (100,0)
p12 = (0,100)
p13 = (250,550)
p14 = (350,150)
p15 = (450,450)
poly1 = [p11, p12, p13, p14, p15]

p21 = (100,0)
p22 = (0,100)
p23 = (200,100)
p24 = (100,200)
p25 = (90,50)
p26 = (300,300)
p27 = (400,0)
poly2 = [p21, p22, p23, p24, p25, p24, p25, p26, p27]
poly0m = tri_polygone(poly0)
poly1m = tri_polygone(poly1)
poly2m = tri_polygone(poly2)

draw_polygon(canvas, poly0m)
draw_polygon(canvas, poly1m)
draw_polygon(canvas, poly2m)

canvas.pack(padx=10, pady=10)

window.mainloop()

Should give the output:
![Ouput of above example run](images/polygons.jpg)

## Voronoi Diagrams
A voronoi diagram can be defined as a set of germs (points in space), which can be associated with a set of cells (polygons). Therefore, a voronoi diagram is implemented as:

`voronoi_diagram = [ (germ0, cell0), (germ1, cell1), ... ] `

In [6]:
voronoi_diagram = [
  ((351, 205),[ (214.23833962665532, 253.1386746795724),
                (299.54702329594477, 275.4167385677308),
                (408.01388888888886, 175.62722222222226),
                (393.6515397082658, 106.11345218800648),
                (366.34635483752254, 92.67307621536082) ]),
  ((541, 104),[]),
  ((449, 460),[]),
  ((255, 114),[]),
  ((496,  65),[]),
  ((498, 296),[]),
  ((479, 542),[]),
  ((186, 569),[]),
  ((375, 524),[]),
  ((236, 448),[ (175.11498194945847, 493.6714801444043),
                (278.0491051942384, 536.206241815801),
                (300.4867582148112, 495.1689553702796),
                (298.96055776892433, 457.0139442231076),
                (213.7531365313653, 383.2767527675277) ]),
  ((  3, 566),[]),
  ((446,  12),[]),
  ((310, 362),[ (183.6344062635929, 259.92583732057415),
                (196.76223972988186, 294.7824985931345),
                (328.3383541513593, 407.00918442321824),
                (412.731848983543, 353.8725395288802),
                (402.7564102564102, 325.45765345765335),
                (299.54702329594477, 275.4167385677308),
                (214.23833962665526, 253.1386746795724) ]),
  ((374, 230),[ (299.54702329594477, 275.4167385677308),
                (402.7564102564102, 325.4576534576536),
                (444.4714240606252, 247.08399115882537),
                (408.01388888888886, 175.62722222222226) ]),
  ((496, 195),[]),
  ((281, 396),[ (196.7622397298818, 294.7824985931345),
                (213.75313653136533, 383.2767527675277),
                (298.9605577689243, 457.01394422310756),
                (328.3383541513593, 407.00918442321824) ]),
  ((117,  69),[]),
  ((361, 443),[ (298.9605577689243, 457.0139442231066),
                (300.4867582148112, 495.1689553702796),
                (399.8827285921625, 477.9894049346881),
                (422.162037340321, 362.6612184736323),
                (412.73184898354305, 353.8725395288803),
                (328.3383541513592, 407.00918442321824) ]),
  ((156, 420),[]),
  ((472, 180),[ (393.6515397082658, 106.11345218800648),
                (408.01388888888886, 175.62722222222214),
                (444.4714240606252, 247.08399115882537),
                (447.13283208020044, 246.4874686716792),
                (510.28803777544596, 145.43913955928645),
                (485.326705940108, 122.77687776141386),
                (400.44403303902016, 105.06223298205637) ])
  ]

In [7]:
def draw_voronoi(canvas, diagram):
    for i in range(0, len(diagram)):
        cell = diagram[i][1]
        germ = canvas_point(diagram[i][0])
        draw_polygon(canvas, cell)
        canvas.create_text(germ[0], germ[1], text="P" + str(i))

window = tk.Tk()

canvas = tk.Canvas(window, width=620, height=620)
draw_cartesian(canvas)

draw_voronoi(canvas, voronoi_diagram)
canvas.pack(padx=10, pady=10)

window.mainloop()

Output window: ![Vornoi example output](images/voronoi_ex.jpg)
Going forward, we will be using code from antoher library from this project, `graphic_library.py`. It has the following functions:

 - `belongs(diagram, point)`: Takes a voronoi diagram and a 2D point to be inserted, and returns the cell to which the point belongs to.
 - `perp_bisector(point1, point2)`: Returns the perpendicular bisector between two point, in the form of `(slope, point)`.
 - `intersection_segment(line, point1, point2)`: returns the point of intersection between `line` and the line formed by `point1` and `point2`. Returns `None` if the two lines do not intersect.
 
 ### Insertion of new germ into Voronoi diagram
 Adding a new germ to the voronoi diagram involves the following steps:
 1. Determine the cell $C_p$ that the new germ $P_{n+1}$ belongs to
 2. Determine the perpendicular bisector between $P_{n+1}$ and germ $P_p$ of the cell $C_p$
 3. Determine the intersection points $S_0$ and $S_1$ of the bisector with the borders of the cell $C_p$ and the ends $a_0$ and $a_1$
 4. Add $S_0$ in the list of vertices of cells $C_{n+1}$ and $C_p$
 5. Add $S_1$ in the list of vertices of cell $C_p$
 6. Determine the germ $P_i$ of cell linked to $C_p$ by the end $a_1$
 7. Remove from $C_p$ the vertices between $S_0$ and $S_1$ that fall inside the new cell $C_{n+1}$
 8. Loop to step 2 with the cell $C_i$ and continue until back to $S_0$

In [8]:
import graphic_library as gl

In [9]:
def find_intersection(diagram, cellNumber, bisection):
    '''Returns the two ends of the cell that cuts the perpendicular bisector
    and the coordinates of the points of intersection'''
    diagram[cellNumber] = (diagram[cellNumber][0], tri_polygone(diagram[cellNumber][1]))
    cell = diagram[cellNumber][1]
    result = []
    for i in range(0, len(cell)):
        p1i = i % len(cell)
        p2i = (i + 1) % len(cell)
        s = gl.intersection_segment(bisection, cell[p1i], cell[p2i])
        if s != None:
            a0, a1 = tri_polygone([cell[p1i], cell[p2i]])
            result.append({'a':((a0[0], a0[1]),(a1[0], a1[1])), 's':(s[0], s[1])})
    return sorted(result, key=lambda i: i['s'][0])

# to test find_intersection()
P12 = (310, 362)
P20 = (290, 340)
print(gl.belongs(voronoi_diagram,P20))
m = gl.perp_bisector(P12,P20)
print(m)
inter = find_intersection(voronoi_diagram,12,m)
print(inter)

window = tk.Tk()

canvas = tk.Canvas(window, width=620, height=620)
draw_cartesian(canvas)

draw_voronoi(canvas, voronoi_diagram)
canvas.pack(padx=10, pady=10)
a0_0 = canvas_point(inter[1]['a'][0])
a0_1 = canvas_point(inter[1]['a'][1])
s0 = canvas_point(inter[1]['s'])

canvas.create_text(a0_0[0], a0_0[1], text='a0')
canvas.create_text(a0_1[0], a0_1[1], text='a0')
canvas.create_text(s0[0], s0[1], text='s0')

window.mainloop()

12
((22, -20), (300.0, 351.0))
[{'a': ((196.76223972988186, 294.7824985931345), (328.3383541513593, 407.00918442321824)), 's': (281.9309559939302, 367.4264036418817)}, {'a': ((299.54702329594477, 275.4167385677308), (402.7564102564102, 325.45765345765335)), 's': (354.0652173913043, 301.84980237154144)}]


In [10]:
def find_neighbor(diagram, cellNumber, edge):
    point = diagram[cellNumber][0]
    ligne = ((edge[1][0] - edge[0][0], edge[1][1] - edge[0][1]), (edge[0][0], edge[0][1]))
    return gl.belongs(diagram, gl.symmetric_point(ligne, point))

def same_point(p1, p2):
    return (sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2) <= gl.EPSILON)

In [11]:
P12 = (310, 362)
P20 = (290, 340)
m = gl.perp_bisector(P12,P20)
inter = find_intersection(voronoi_diagram,12,m)
print(find_neighbor(voronoi_diagram,12,inter[0]['a']))
print(find_neighbor(voronoi_diagram,12,inter[1]['a']))

15
13


In [12]:
def keep_good_points(poly, S0, S1, Pn1, Pp):
    L = [S0, S1]
    for p in poly:
        if p != S0 and p != S1 and gl.carre_distance(p,Pn1) >= gl.carre_distance(p,Pp):
            L.append(p)
    T = tri_polygone(L)
    return T

def insert_germ(diagram, point):
    cellNumber = gl.belongs(diagram, point)
    diagram.append((point, []))
    Pp = diagram[cellNumber][0]
    bisection = gl.perp_bisector(Pp, point)
    inter = find_intersection(diagram, cellNumber, bisection)
    firstS0 = inter[0]['s']

    S0 = inter[0]['s']
    S1 = inter[1]['s']
    a1 = inter[1]['a']

    diagram[-1][1].append(S0)
    diagram[cellNumber][1].append(S0)
    diagram[cellNumber][1].append(S1)

    PiNumber = find_neighbor(diagram, cellNumber, a1)

    temp = diagram[cellNumber][1].copy()
    diagram[cellNumber] = (diagram[cellNumber][0], keep_good_points(temp, S0, S1, point, Pp))

    while not same_point(S1, firstS0):
        Pp = diagram[PiNumber][0]

        bisection = gl.perp_bisector(Pp, point)
        inter = find_intersection(diagram, PiNumber, bisection)

        newS0 = inter[0]['s']
        newS1 = inter[1]['s']

        if same_point(S0, newS0) or same_point(S1, newS1):
            S0 = newS1
            S1 = newS0
            a1 = inter[0]['a']
        else:
            S0 = newS0
            S1 = newS1
            a1 = inter[1]['a']

        diagram[-1][1].append(S0)
        diagram[PiNumber][1].append(S0)
        diagram[PiNumber][1].append(S1)

        newPiNumber = find_neighbor(diagram, PiNumber, a1)

#         temp = diagram[PiNumber][1].copy()
#         diagram[PiNumber] = (diagram[PiNumber][0], keep_good_points(temp, S0, S1, point, Pp))
        PiNumber = newPiNumber

In [13]:
diagram_test = [((150.0, 150.0), [(0, 0), (300.0, 0.0), (300.0, 262.5), (-0.0, 412.5)]),
                        ((450.0, 150.0), [(300.0, 0.0), (600, 0), (600.0, 412.5), (300.0, 262.5)]),
                        ((300.0, 450.0), [(300.0, 262.5), (600.0, 412.5), (-0.0, 412.5), (0, 600), (600, 600)])]

insert_germ(diagram_test,(300,300))

window = tk.Tk()
canvas = tk.Canvas(window, width=620, height=620)
draw_cartesian(canvas)
draw_voronoi(canvas,diagram_test)
canvas.pack(padx=10, pady=10)
window.mainloop()

Output for test diagram:
![Ouput of test for germ insert](images/test_insert.jpg)

In [14]:
insert_germ(voronoi_diagram, (315, 400))
window = tk.Tk()
canvas = tk.Canvas(window, width=620, height=620)
draw_cartesian(canvas)
draw_voronoi(canvas,voronoi_diagram)
canvas.pack(padx=10, pady=10)
window.mainloop()

Ouput after insert:
![Ouput after inserting new germ](images/voronoi_insert.jpg)

At this instant, the `germ_insert()` function stops when it finds the point $S_0$ again. However, this can cause problems if the new cell to be constructed is at the border of the canvas. In such a case, it is necessary to capture the corners of the new cell, characterized by the points which are closer to the new cell than the previous cell.

Below, the function `is_border()` tests if the point to be added is a border case. Subsequently, the function `capture_corners()` takes the new germ and returns the list of corners (among the 4 corners of the canvas) which are the closest to it. We then redefine `insert_germ()` to add this capability. This requires solving three problems:
1. Once we reach a border, we need to restart from $S_0$, circling in the opposite direction this time. Therefore we need to memorize the information relative to $S_0$ at the beggining of the function.
2. If we reach a border a second time, we must stop. For this, we can keep a boolean to distinguish between the two situations.
3. Capture the corners belonging to the new cell.

In [15]:
def is_border(point):
    return (abs(canvas_len - point[0]) <= gl.EPSILON) or \
           (abs(point[0]) <= gl.EPSILON) or \
           (abs(canvas_len - point[1]) <= gl.EPSILON) or \
           (abs(point[1]) <= gl.EPSILON)

def capture_corners(seed):
    L = [(0, 0),(0, canvas_len),(canvas_len, 0),(canvas_len, canvas_len)]
    return sorted(L, key= lambda p: sqrt((p[0]-seed[0])**2 + (p[1]-seed[1])**2))

In [16]:
def insert_germ_v2(diagram, point):
    '''NOT FULLY IMPLEMENTED YET!'''
    secondBorder = False
    cellNumber = gl.belongs(diagram, point)
    diagram.append((point, []))
    Pp = diagram[cellNumber][0]
    bisection = gl.perp_bisector(Pp, point)
    inter = find_intersection(diagram, cellNumber, bisection)

    firstS0 = inter[0]['s']
    firsta0 = inter[0]['a']
    firstS1 = inter[1]['s']

    S0 = inter[0]['s']
    S1 = inter[1]['s']
    a1 = inter[1]['a']
    
    if(is_border(S1) and is_border(S0)):
        coins = capture_corners(point)
        
        diagram[-1][1].append(coins[0])
        diagram[-1][1].append(S1)
        diagram[-1][1].append(S0)
        diagram[-1] = (diagram[-1][0], tri_polygone(diagram[-1][1]))

        temp = diagram[cellNumber][1].copy()
        diagram[cellNumber] = (diagram[cellNumber][0], keep_good_points(temp, S0, S1, point, Pp))
        return

    if(is_border(S1)):
        S0, S1 = S1, S0
        a1, firsta0 = firsta0, a1
        secondBorder = True
        coins = capture_corners(point)
        diagram[-1][1].append(coins[0])
        diagram[-1][1].append(S1)

    diagram[-1][1].append(S0)
    diagram[cellNumber][1].append(S0)
    diagram[cellNumber][1].append(S1)

    PiNumber = find_neighbor(diagram, cellNumber, a1)

    temp = diagram[cellNumber][1].copy()
    diagram[cellNumber] = (diagram[cellNumber][0], keep_good_points(temp, S0, S1, point, Pp))

    while not same_point(S1, firstS0):
        Pp = diagram[PiNumber][0]

        bisection = gl.perp_bisector(Pp, point)
        inter = find_intersection(diagram, PiNumber, bisection)
#         if len(inter)==0:
#             break
        print(point, PiNumber, inter)
        if secondBorder:
            newS0 = inter[1]['s']
            newS1 = inter[0]['s']
        else:
            newS0 = inter[0]['s']
            newS1 = inter[1]['s']

        if same_point(S0, newS0) or same_point(S1, newS1):
            S0 = newS1
            S1 = newS0
            a1 = inter[0]['a']
        else:
            S0 = newS0
            S1 = newS1
            a1 = inter[1]['a']

        diagram[-1][1].append(S0)
        diagram[PiNumber][1].append(S0)
        diagram[PiNumber][1].append(S1)

        if not is_border(S1) and not is_border(S0):
            newPiNumber = find_neighbor(diagram, PiNumber, a1)

            temp = diagram[PiNumber][1].copy()
            diagram[PiNumber] = (diagram[PiNumber][0], keep_good_points(temp, S0, S1, point, Pp))
            PiNumber = newPiNumber
        else:
            if not secondBorder:
                print('here')
                secondBorder = True
                coins = capture_corners(point)
#                 diagram[-1][1].append(S1) 
                diagram[-1][1].append(coins[0])
                S0, S1 = firstS0, firstS1
                a1= firsta0
                
                newPiNumber = find_neighbor(diagram, PiNumber, firsta0)

                temp = diagram[PiNumber][1].copy()
                diagram[PiNumber] = (diagram[PiNumber][0], keep_good_points(temp, S0, S1, point, Pp))
                PiNumber = newPiNumber
            else:
                print('here2')
                coins = capture_corners(S0)
                S1 = coins[0]
                diagram[-1][1].append(S1)
                diagram[-1] = (diagram[-1][0], tri_polygone(diagram[-1][1]))
                return

In [17]:
list_seeds = [cell[0] for cell in voronoi_diagram]
test_diagram = [ ( list_seeds[0], [  (-10, -10), (-10,610), (610,-10), (610,610) ] ) ]

insert_germ(test_diagram, list_seeds[1])
insert_germ(test_diagram, list_seeds[2])
insert_germ(test_diagram, list_seeds[3])

window = tk.Tk()

canvas = tk.Canvas(window, width=620, height=620)

draw_cartesian(canvas)

draw_voronoi(canvas,test_diagram)

canvas.pack(padx=10, pady=10)

window.mainloop()

IndexError: list index out of range